<a href="https://colab.research.google.com/github/xhavien/CCMACLRL_EXAM/blob/main/CCMACLRL_MIDTERMEXAM_Pagdanganan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor

In [2]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)

In [3]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [4]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [5]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [6]:
sf

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016
3,188536,43878.016
4,188537,43878.016
...,...,...
125685,314218,43878.016
125686,314219,43878.016
125687,314220,43878.016
125688,314221,43878.016


In [7]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [8]:
df['fuel_type'].value_counts()

,count
fuel_type,
Gasoline,165940
Hybrid,6832
E85 Flex Fuel,5406
Diesel,3955
–,781
Plug-In Hybrid,521
not supported,15


In [9]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
58501,58501,Rolls-Royce,Cullinan,2020,10654,Gasoline,6.7L V12 48V GDI DOHC Twin Turbo,8-Speed Automatic,Red,Black,None reported,Yes,41999
46379,46379,BMW,328 i,2013,112465,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,White,Black,At least 1 accident or damage reported,Yes,12500
19669,19669,Chevrolet,Corvette Stingray w/2LT,2020,74422,Gasoline,6.2L V8 16V GDI OHV,8-Speed Automatic,Silver Ice Metallic,Black,None reported,Yes,36994
19611,19611,Kia,Stinger GT2,2016,69100,Gasoline,365.0HP 3.3L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,Yes,30500
131121,131121,Land,Rover Range Rover P525 Westminster,2022,15042,Gasoline,5.0L V8 32V GDI DOHC Supercharged,8-Speed Automatic,Silver,Ebony,None reported,Yes,61900
124357,124357,Hummer,H3 Base,2007,165454,Gasoline,242.0HP 3.7L 5 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,At least 1 accident or damage reported,Yes,4000
102378,102378,BMW,M6 Gran Coupe Base,2017,40200,Gasoline,560.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Blue,Black,At least 1 accident or damage reported,Yes,42950
23303,23303,Ford,Explorer ST,2020,96000,Gasoline,290.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,24500
131342,131342,Nissan,Titan SL,2019,69000,Gasoline,390.0HP 5.6L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Gray,None reported,Yes,35000
103801,103801,Lexus,IS 300 Base,2010,106500,Gasoline,255.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Silver,Gray,At least 1 accident or damage reported,Yes,8250


In [10]:
print(len(df))

188533


In [11]:
df['clean_title'].value_counts()

,count
clean_title,
Yes,167114


In [12]:
df['clean_title'] = df['clean_title'].fillna('No')
dt['clean_title'] = df['clean_title'].fillna('No')

In [13]:
df['clean_title'].value_counts()

,count
clean_title,
Yes,167114
No,21419


In [14]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
110209,110209,Mercedes-Benz,CLA-Class CLA 250,2014,156375,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,8900
119060,119060,Mazda,MX-5 Miata Grand Touring,2013,134000,Gasoline,167.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Silver,Black,None reported,Yes,11000
119422,119422,Ford,F-150 Lariat,2022,57000,Gasoline,400.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Black,None reported,Yes,57500
8396,8396,Audi,Q5 2.0T Premium Plus,2018,125000,Gasoline,211.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,60999
172049,172049,Subaru,BRZ Limited,2015,72329,Gasoline,180.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Red,None reported,Yes,15900
94389,94389,Tesla,Model S Performance,2018,100312,NaN,Electric,1-Speed Automatic,White,Black,None reported,No,25884
62940,62940,Volkswagen,Golf GTI 2.0T SE 4-Door,2016,126500,Gasoline,180.0HP 1.8L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Black,None reported,Yes,17000
147401,147401,Buick,Rainier CXL,2004,170000,Gasoline,275.0HP 4.2L Straight 6 Cylinder Engine Gasoli...,A/T,White,Beige,At least 1 accident or damage reported,Yes,6399
101587,101587,GMC,Sierra 2500 Denali,2017,86000,Gasoline,401.0HP 6.6L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,39500
113224,113224,GMC,Acadia SLE-2,2020,9642,Gasoline,3.0L V6 24V GDI DOHC Turbo,8-Speed Automatic,Summit White,Jet Black,None reported,No,52999


In [15]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [16]:
df['accident'].value_counts()

,count
accident,
None reported,144514
At least 1 accident or damage reported,41567


In [17]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [18]:
print(len(df))

188533


In [19]:
# Car Age

from datetime import datetime

current_year = datetime.now().year
df['car age'] = current_year - df['model_year']
dt['car age'] = current_year - df['model_year']


In [20]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3


In [21]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,car age
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,17
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes,22
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,Yes,22
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,Yes,7
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,3


In [22]:
print(len(df))

188533


In [23]:
print(len(dt))

125690


In [24]:
dt.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [25]:
# For df dataframe
df['liter'] = pd.to_numeric(df['engine'].str.extract(r'(?P<liter>\d+\.?\d*)\s*(?:L|Liter)')['liter'], errors='coerce')

# For dt dataframe
dt['liter'] = pd.to_numeric(dt['engine'].str.extract(r'(?P<liter>\d+\.?\d*)\s*(?:L|Liter)')['liter'], errors='coerce')


In [26]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])
dt['brand'] = label_encoder.fit_transform(dt['brand'])

In [27]:
label_encoder = LabelEncoder()
df['model'] = label_encoder.fit_transform(df['model'])
dt['model'] = label_encoder.fit_transform(dt['model'])


In [28]:
label_encoder = LabelEncoder()
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
dt['fuel_type'] = label_encoder.fit_transform(dt['fuel_type'])

In [29]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age,liter
0,0,31,495,2007,213000,2,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17,1.6
1,1,28,930,2002,143250,2,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22,3.9
2,2,9,1575,2002,136731,1,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22,5.3
3,3,16,758,2017,19500,2,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7,5.0
4,4,36,1077,2021,7388,2,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3,2.0


In [30]:
df['accident'].value_counts()

,count
accident,
None reported,144514
At least 1 accident or damage reported,41567


In [31]:

df['accident'] = label_encoder.fit_transform(df['accident'])

dt['accident'] = label_encoder.fit_transform(dt['accident'])

In [32]:
df['clean_title'] = label_encoder.fit_transform(df['clean_title'])
dt['clean_title'] = label_encoder.fit_transform(dt['clean_title'])

In [33]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [34]:
label_encoder = LabelEncoder()
df['transmission'] = label_encoder.fit_transform(df['transmission'])
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])

In [35]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age,liter
0,0,31,495,2007,213000,2,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,38,Yellow,Gray,1,1,4200,17,1.6
1,1,28,930,2002,143250,2,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,38,Silver,Beige,0,1,4999,22,3.9
2,2,9,1575,2002,136731,1,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,38,Blue,Gray,1,1,13900,22,5.3
3,3,16,758,2017,19500,2,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,49,Black,Black,1,1,45000,7,5.0
4,4,36,1077,2021,7388,2,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,23,Black,Beige,1,1,97500,3,2.0


In [36]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,car age,liter
0,188533,26,1388,2015,98000,2,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,16,White,Beige,1,1,17,2.0
1,188534,26,1375,2020,9142,3,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,31,Silver,Black,1,1,22,3.0
2,188535,14,636,2022,28121,2,3.5L V6 24V PDI DOHC Twin Turbo,3,White,Ebony,1,1,22,3.5
3,188536,3,182,2016,61258,2,2.0 Liter TFSI,39,Silician Yellow,Black,1,1,7,2.0
4,188537,3,181,2018,59000,2,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,38,Gray,Black,1,1,3,2.0


In [37]:
mean_imputer = SimpleImputer(strategy='mean')
df['liter'] = mean_imputer.fit_transform(df[['liter']])
dt['liter'] = mean_imputer.fit_transform(dt[['liter']])

In [38]:
df.drop(columns = ['id', 'engine', 'ext_col', 'int_col'], inplace=True)
dt.drop(columns = ['id', 'engine', 'ext_col', 'int_col'], inplace=True)

In [39]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,transmission,accident,clean_title,price,car age,liter
36217,36,550,2018,56837,2,36,1,1,26299,6,2.0
98342,20,1262,2018,16100,2,38,0,1,28000,6,3.5
93811,11,1341,2010,149460,2,38,0,1,6800,14,3.7
21025,36,741,2021,77648,2,49,1,1,56000,3,4.0
45473,28,1142,2022,30426,2,32,1,0,46120,2,2.7


In [40]:
dt.sample(5)

,brand,model,model_year,milage,fuel_type,transmission,accident,clean_title,car age,liter
4310,4,68,2020,7758,2,39,1,1,15,2.0
10098,31,495,2012,66000,2,16,1,1,5,1.6
38955,3,185,2016,89987,2,49,1,1,5,3.0
118581,52,1213,2014,106000,2,38,0,1,2,1.8
3039,4,103,2006,135000,2,16,2,1,20,3.0


In [41]:
from sklearn.model_selection import train_test_split

y = df["price"]



x = df.drop(['price'], axis = 1)



X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2, random_state=42)

In [42]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [43]:
y_pred_train = linear_reg.predict(X_train)
y_pred_test = linear_reg.predict(X_test)

In [44]:
train_mse = mean_squared_error(y_train, y_pred_train, squared=False)
test_mse = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Training MSE: {train_mse}")
print(f"Testing MSE: {test_mse}")
print(f"Training R2 Score: {train_r2}")
print(f"Testing R2 Score: {test_r2}")

Training MSE: 76109.32272585986
Testing MSE: 70588.53965097434
Training R2 Score: 0.09139828499385938
Testing R2 Score: 0.10400643520268893


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [45]:
id = sf.pop('id')
y_pred = linear_reg.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'price': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('results.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
